In [7]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, execute, BasicAer, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService

# Define Gates

### Reverse CNOT

In [10]:
"""
Input: a, b

Output: 
P = a+b
Q = b
"""

c = QuantumCircuit(2)
c.h(0)
c.h(1)
c.cx(0,1)
c.h(0)
c.h(1)
revcx = c.to_gate(label="rev_CNOT")
c.draw()

┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├
     ├───┤┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘

### Peres Gate

In [11]:
"""
Input: a, b, c

Output:
P = a
Q = a + b
R = a*b + c
"""
c = QuantumCircuit(3)
#c.reset(0)
#c.reset(1)
#c.reset(2)
#c.x(0)
#c.x(1)
#c.x(2)
c.ccx(0,1,2)
c.cx(0,1)
peres = c.to_gate(label="Peres")
c.draw()

q_0: ──■────■──
       │  ┌─┴─┐
q_1: ──■──┤ X ├
     ┌─┴─┐└───┘
q_2: ┤ X ├─────
     └───┘

In [12]:
#c.measure_all()
# Use Aer's qasm_simulator
simulator = Aer.get_backend('statevector_simulator')

# Execute the circuit on the qasm simulator
job = simulator.run(c)

# Grab results from the job
result = job.result()

# Return counts
outputstate = result.get_statevector(c, decimals=3)
print(outputstate)

Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


### Full Adder

In [13]:
"""
Input: 
a = 1st operand, 
b = 2nd operand, 
c = 0, 
d = c_in

Output: 
P = garbage, 
Q = garbage, 
R = sum, 
T = carry
"""
c = QuantumCircuit(4)
c.append(peres, [0,1,2])
c.swap(2,3)
c.append(peres, [1,2,3])
fa = c.to_gate(label="RFA")
c.draw()

┌────────┐             
q_0: ┤0       ├─────────────
     │        │   ┌────────┐
q_1: ┤1 Peres ├───┤0       ├
     │        │   │        │
q_2: ┤2       ├─X─┤1 Peres ├
     └────────┘ │ │        │
q_3: ───────────X─┤2       ├
                  └────────┘

### Fredkin

In [14]:
"""
By considering only the output qubit Q and by using input qubits 'a' as the selection line, Fredkin gate can 
be used as a 2x1 MUX.

Input: a, b, c

Output:
P = a,
Q = a'*b + a*c
R = a*b + a'*c
"""
c = QuantumCircuit(3)
c.append(revcx, [1,2])
c.ccx(0,1,2)
c.append(revcx, [1,2])
fredkin = c.to_gate(label="FG")
c.draw()

q_0: ───────────────■───────────────
     ┌───────────┐  │  ┌───────────┐
q_1: ┤0          ├──■──┤0          ├
     │  rev_CNOT │┌─┴─┐│  rev_CNOT │
q_2: ┤1          ├┤ X ├┤1          ├
     └───────────┘└───┘└───────────┘

# CSA Design-1

Each stage of the CSA can be designed as a quantum gate as follows:

In [41]:
"""
This circuit represents a single stage of the CSA.

Input:
a = first operand, represents "a"
b = second operand, represents "b"
c = 0, ancilla, represents "a"
d = 0, ancilla, represents "b"
e = 0, represents "c=0"
f = 1, represents "c=1"
g = 0, ancilla
h = 0, ancilla
i = 0 or 1
l = 0, ancilla

Output:
P = g
Q = g
R = g
S = g
T = carry
U = g
V = sum
W = g
X = carry
Y = g

5 ancilla bits and 7 garbage bits (in contrast with 8 needed in the paper)
"""

a_in = QuantumRegister(1, name="a")
b_in = QuantumRegister(1, name="b")
a_copies = QuantumRegister(1, name="a'")
b_copies = QuantumRegister(1, name="b'")
carries = QuantumRegister(2, name="c")
zero = QuantumRegister(2, name="zero")
c_in = QuantumRegister(2, name="c_in")

csa = QuantumCircuit(a_in, b_in, a_copies, b_copies, carries, zero, c_in)
csa.reset(a_copies[0])
csa.reset(b_copies[0])
csa.cx(a_in[0], a_copies[0])                                        # a' = a
csa.cx(b_in[0], b_copies[0])                                        # b' = b
csa.reset(carries[0])                                               # c = 0
csa.reset(carries[1])
csa.x(carries[1])                                                   # c = 1
csa.reset(zero[0])
csa.reset(zero[1])
csa.reset(c_in[1])                                                  # c_in' = c_in

csa.cx(c_in[0], c_in[1])
csa.append(fa, [a_in[0], b_in[0], zero[0], carries[0]])
csa.append(fa, [a_copies[0], b_copies[0], zero[1], carries[1]])
csa.append(fredkin, [c_in[1], zero[0], zero[1]])                    # mux for the sum qubits
csa.append(fredkin, [c_in[0], carries[0], carries[1]])              # mux for the carry qubits

csa.reset(c_in[0])                                                  # Recycle c_in for the output of buffers

csa.cx(carries[0], c_in[0])                                         # Buffer

csa.draw()

┌──────┐                                
     a: ───────■───────┤0     ├────────────────────────────────
               │       │      │                                
     b: ───────┼────■──┤1     ├────────────────────────────────
             ┌─┴─┐  │  │      │┌──────┐                        
    a': ─|0>─┤ X ├──┼──┤      ├┤0     ├────────────────────────
             └───┘┌─┴─┐│      ││      │                        
    b': ─|0>──────┤ X ├┤  RFA ├┤1     ├────────────────────────
                  └───┘│      ││      │       ┌─────┐          
   c_0: ─|0>───────────┤3     ├┤      ├───────┤1    ├───────■──
             ┌───┐     │      ││  RFA │       │     │       │  
   c_1: ─|0>─┤ X ├─────┤      ├┤3     ├───────┤2    ├───────┼──
             └───┘     │      ││      │┌─────┐│     │       │  
zero_0: ─|0>───────────┤2     ├┤      ├┤1    ├┤  FG ├───────┼──
                       └──────┘│      ││     ││     │       │  
zero_1: ─|0>───────────────────┤2     ├┤2    ├┤     ├───────┼──
                               └──────┘│  FG ││     │     ┌─┴─┐
c_in_0: ───────■───────────────────────┤     ├┤0    ├─|0>─┤ X ├
             ┌─┴─┐                     │     │└─────┘     └───┘
c_in_1: ─|0>─┤ X ├─────────────────────┤0    ├─────────────────
             └───┘                     └─────┘

Note that although the logic gates are represented in series, the circuit will be re-arranged by the transpiler in order to execute gates in parallel where possible. For instance, if you swap the position of "b" with "a'" it is clear that the two CNOTs at the beginning of the circuit can be run simultaneously.

The quantum logic gate CSA has number of input and output of 10 and we have different options to combine the stages together:

A first approach is by remarking the classical CSA exploiting the fact that all the FA operations can be done in parallel, but that means using a lot of qubits. As a matter of fact, if we want to make a two stage CSA, we have to connect the input qubits "i" and "l" (the carries propagated by the first stage) of the second CSA to the output T and X of the first one. To force the parallel execution of the four RFAs instead, we have to use 8 qubits for the first stage and 8 more qubits for the second stage. Therefore the total number of qubits used to implement the circuit is 8n+2. It is clear that simulating a 4-qubit CSA is not possible.

Another approach is to run each stage in series, reducing the number of qubits used to 10, but in this way the CSA looses all its advantages.

## Iterative construction of Design-1

Based on the discussion above, here's an iterative fucntion to build CSAs operating on n-qubits numbers. Measurments and output registers to store classical bits are added to test the circuit.

In [67]:
def generate_CSA_n_qubits(n):
    a_in = QuantumRegister(n, name="a")
    b_in = QuantumRegister(n, name="b")
    a_copies = QuantumRegister(n, name="a'")
    b_copies = QuantumRegister(n, name="b'")
    carries = QuantumRegister(n, name="c")
    carries_copies = QuantumRegister(n, name="c'")
    zero = QuantumRegister(n, name="zero")
    zero_copies = QuantumRegister(n, name="zero'")
    c_in = QuantumRegister(2, name="c_in")

    output = ClassicalRegister(n+1, name='output')

    csa = QuantumCircuit(a_in, b_in, a_copies, b_copies, carries, carries_copies, zero, zero_copies, c_in, output)
    
    for i in range(n):
        csa.reset(a_copies[i])
        csa.reset(b_copies[i])
        csa.cx(a_in[i], a_copies[i])                                        # a' = a
        csa.cx(b_in[i], b_copies[i])                                        # b' = b
        csa.reset(carries[i])                                               # c = 0
        csa.reset(carries_copies[i])
        csa.x(carries_copies[i])                                                   # c = 1
        csa.reset(zero[i])
        csa.reset(zero_copies[i])
        csa.reset(c_in[1])                                                  # c_in' = c_in

        csa.cx(c_in[0], c_in[1])
        csa.append(fa, [a_in[i], b_in[i], zero[i], carries[i]])
        csa.append(fa, [a_copies[i], b_copies[i], zero_copies[i], carries_copies[i]])
        csa.append(fredkin, [c_in[1], zero[i], zero_copies[i]])                    # mux for the sum qubits
        csa.append(fredkin, [c_in[0], carries[i], carries_copies[i]])              # mux for the carry qubits

        csa.measure(zero[i], output[i])

        csa.reset(c_in[0])                                                  # Recycle c_in for the output of buffers
        if (i < n-1):
            csa.cx(carries[i], c_in[0])                                         # Buffer
        else:
            csa.measure(carries[i], output[i+1])
    return csa

In [69]:
csa2 = generate_CSA_n_qubits(2)
csa2.draw()

┌──────┐                               »
     a_0: ───────■─────────────────┤0     ├───────────────────────────────»
                 │                 │      │        ┌──────┐               »
     a_1: ───────┼─────────■───────┤      ├────────┤0     ├───────────────»
                 │         │       │      │        │      │               »
     b_0: ───────┼────■────┼───────┤1     ├────────┤      ├───────────────»
                 │    │    │       │      │        │      │               »
     b_1: ───────┼────┼────┼────■──┤      ├────────┤1     ├───────────────»
               ┌─┴─┐  │    │    │  │      │┌──────┐│      │               »
    a'_0: ─|0>─┤ X ├──┼────┼────┼──┤      ├┤0     ├┤      ├───────────────»
               └───┘  │  ┌─┴─┐  │  │      ││      ││      │┌──────┐       »
    a'_1: ─|0>────────┼──┤ X ├──┼──┤      ├┤      ├┤      ├┤0     ├───────»
                    ┌─┴─┐└───┘  │  │      ││      ││      ││      │       »
    b'_0: ─|0>──────┤ X ├───────┼──┤  RFA ├┤1     ├┤      ├┤      ├───────»
                    └───┘     ┌─┴─┐│      ││      ││      ││      │       »
    b'_1: ─|0>────────────────┤ X ├┤      ├┤      ├┤  RFA ├┤1     ├───────»
                              └───┘│      ││      ││      ││      │       »
     c_0: ─|0>─────────────────────┤3     ├┤      ├┤      ├┤      ├───────»
                                   │      ││      ││      ││      │       »
     c_1: ─|0>─────────────────────┤      ├┤  RFA ├┤3     ├┤      ├───────»
               ┌───┐               │      ││      ││      ││      │       »
    c'_0: ─|0>─┤ X ├───────────────┤      ├┤3     ├┤      ├┤  RFA ├───────»
               ├───┤               │      ││      ││      ││      │       »
    c'_1: ─|0>─┤ X ├───────────────┤      ├┤      ├┤      ├┤3     ├───────»
               └───┘               │      ││      ││      ││      │┌─────┐»
  zero_0: ─|0>─────────────────────┤2     ├┤      ├┤      ├┤      ├┤1    ├»
                                   └──────┘│      ││      ││      ││     │»
  zero_1: ─|0>─────────────────────────────┤      ├┤2     ├┤      ├┤     ├»
                                           │      │└──────┘│      ││     │»
 zero'_0: ─|0>─────────────────────────────┤2     ├────────┤      ├┤2    ├»
                                           └──────┘        │      ││  FG │»
 zero'_1: ─|0>─────────────────────────────────────────────┤2     ├┤     ├»
                                                           └──────┘│     │»
  c_in_0: ───────■─────────────────────────────────────────────────┤     ├»
               ┌─┴─┐                                               │     │»
  c_in_1: ─|0>─┤ X ├───────────────────────────────────────────────┤0    ├»
               └───┘                                               └─────┘»
output: 3/════════════════════════════════════════════════════════════════»
                                                                          »
«                                                            
«     a_0: ──────────────────────────────────────────────────
«                                                            
«     a_1: ──────────────────────────────────────────────────
«                                                            
«     b_0: ──────────────────────────────────────────────────
«                                                            
«     b_1: ──────────────────────────────────────────────────
«                                                            
«    a'_0: ──────────────────────────────────────────────────
«                                                            
«    a'_1: ──────────────────────────────────────────────────
«                                                            
«    b'_0: ──────────────────────────────────────────────────
«                                                            
«    b'_1: ──────────────────────────────────────────────────
«          ┌─────┐                                           
«     c_0: ┤1    

### Transpiling

In [241]:
circ.draw()

┌──────────────────────┐                                       »
     a_0: ────┤0                     ├──────■────────────────────────────────»
              │                      │      │                                »
     a_1: ────┤1                     ├──────┼────■───────────────────────────»
              │  Initialize(1,1,0,0) │      │    │                           »
     a_2: ────┤2                     ├──────┼────┼────■──────────────────────»
              │                      │      │    │    │                      »
     a_3: ────┤3                     ├──────┼────┼────┼────■─────────────────»
              ├──────────────────────┤      │    │    │    │                 »
     b_0: ────┤0                     ├──────┼────┼────┼────┼────■────────────»
              │                      │      │    │    │    │    │            »
     b_1: ────┤1                     ├──────┼────┼────┼────┼────┼────■───────»
              │  Initialize(0,0,1,1) │      │    │    │    │    │    │       »
     b_2: ────┤2                     ├──────┼────┼────┼────┼────┼────┼────■──»
              │                      │      │    │    │    │    │    │    │  »
     b_3: ────┤3                     ├──────┼────┼────┼────┼────┼────┼────┼──»
              └──────────────────────┘    ┌─┴─┐  │    │    │    │    │    │  »
    a'_0: ────────────────────────────────┤ X ├──┼────┼────┼────┼────┼────┼──»
                                          └───┘┌─┴─┐  │    │    │    │    │  »
    a'_1: ─────────────────────────────────────┤ X ├──┼────┼────┼────┼────┼──»
                                               └───┘┌─┴─┐  │    │    │    │  »
    a'_2: ──────────────────────────────────────────┤ X ├──┼────┼────┼────┼──»
                                                    └───┘┌─┴─┐  │    │    │  »
    a'_3: ───────────────────────────────────────────────┤ X ├──┼────┼────┼──»
                                                         └───┘┌─┴─┐  │    │  »
    b'_0: ────────────────────────────────────────────────────┤ X ├──┼────┼──»
                                                              └───┘┌─┴─┐  │  »
    b'_1: ─────────────────────────────────────────────────────────┤ X ├──┼──»
                                                                   └───┘┌─┴─┐»
    b'_2: ──────────────────────────────────────────────────────────────┤ X ├»
                                                                        └───┘»
    b'_3: ───────────────────────────────────────────────────────────────────»
          ┌──────────────────────────────┐                                   »
     c_0: ┤0                             ├───────────────────────────────────»
          │                              │                                   »
     c_1: ┤1                             ├───────────────────────────────────»
          │                              │                                   »
     c_2: ┤2                             ├───────────────────────────────────»
          │                              │                                   »
     c_3: ┤3                             ├───────────────────────────────────»
          │  Initialize(0,1,0,1,0,1,0,1) │                                   »
     c_4: ┤4                             ├───────────────────────────────────»
          │                              │                                   »
     c_5: ┤5                             ├───────────────────────────────────»
          │                              │                                   »
     c_6: ┤6                             ├───────────────────────────────────»
          │                              │                                   »
     c_7: ┤7                             ├───────────────────────────────────»
          └─────┬──────────────────┬─────┘                                   »
  zero_0: ──────┤0                 ├─────────────────────────────────────────»
                │  Initialize(0,0) │                            

In [239]:
usim = Aer.get_backend('unitary_simulator')
transpiled = transpile(circuit, backend=usim)
transpiled.draw()

»
     a_0: ───────■──────────────────────────────────■────■────────────────────»
                 │                                  │    │                    »
     a_1: ───────┼────■─────────────────────────────┼────┼────────────────────»
          ┌───┐  │    │                             │    │                    »
     a_2: ┤ X ├──┼────┼─────────■───────────────────┼────┼────────────────────»
          ├───┤  │    │         │                   │    │                    »
     a_3: ┤ X ├──┼────┼─────────┼────■──────────────┼────┼────────────────────»
          ├───┤  │    │         │    │              │  ┌─┴─┐                  »
     b_0: ┤ X ├──┼────┼─────────┼────┼────■─────────■──┤ X ├──────────■───────»
          ├───┤  │    │         │    │    │         │  └───┘          │       »
     b_1: ┤ X ├──┼────┼─────────┼────┼────┼────■────┼─────────────────┼───────»
          └───┘  │    │         │    │    │    │    │                 │       »
     b_2: ──■────┼────┼─────────┼────┼────┼────┼────┼─────────────────┼───────»
            │    │    │         │    │    │    │    │                 │       »
     b_3: ──┼────┼────┼────■────┼────┼────┼────┼────┼─────────────────┼───────»
            │  ┌─┴─┐  │    │    │    │    │    │    │                 │       »
    a'_0: ──┼──┤ X ├──┼────┼────┼────┼────┼────┼────┼────■────■───────┼───────»
            │  └───┘┌─┴─┐  │    │    │    │    │    │    │    │       │       »
    a'_1: ──┼───────┤ X ├──┼────┼────┼────┼────┼────┼────┼────┼───────┼───────»
            │       └───┘  │  ┌─┴─┐  │    │    │    │    │    │       │       »
    a'_2: ──┼──────────────┼──┤ X ├──┼────┼────┼────┼────┼────┼───────┼───────»
            │              │  └───┘┌─┴─┐  │    │    │    │    │       │       »
    a'_3: ──┼──────────────┼───────┤ X ├──┼────┼────┼────┼────┼───────┼───────»
            │              │       └───┘┌─┴─┐  │    │    │  ┌─┴─┐     │       »
    b'_0: ──┼──────────────┼────────────┤ X ├──┼────┼────■──┤ X ├─────┼────■──»
            │              │            └───┘┌─┴─┐  │    │  └───┘     │    │  »
    b'_1: ──┼──────────────┼─────────────────┤ X ├──┼────┼────────────┼────┼──»
          ┌─┴─┐            │                 └───┘  │    │            │    │  »
    b'_2: ┤ X ├────────────┼────────────────────────┼────┼────────────┼────┼──»
          └───┘          ┌─┴─┐                      │    │            │    │  »
    b'_3: ───────────────┤ X ├──────────────────────┼────┼────────────┼────┼──»
                         ├───┤                      │    │          ┌─┴─┐  │  »
     c_0: ───────────────┤ X ├──────────────────────┼────┼────X─────┤ X ├──┼──»
                         └───┘                      │    │    │     └─┬─┘┌─┴─┐»
     c_1: ──────────────────────────────────────────┼────┼────┼───X───┼──┤ X ├»
                         ┌───┐                      │    │    │   │   │  └─┬─┘»
     c_2: ───────────────┤ X ├──────────────────────┼────┼────┼───┼───┼────┼──»
                         └───┘                      │    │    │   │   │    │  »
     c_3: ──────────────────────────────────────────┼────┼────┼───┼───┼────┼──»
                         ┌───┐                      │    │    │   │   │    │  »
     c_4: ───────────────┤ X ├──────────────────────┼────┼────┼───┼───┼────┼──»
                         └───┘                      │    │    │   │   │    │  »
     c_5: ──────────────────────────────────────────┼────┼────┼───┼───┼────┼──»
                         ┌───┐                      │    │    │   │   │    │  »
     c_6: ───────────────┤ X ├──────────────────────┼────┼────┼───┼───┼────┼──»
                         └───┘                      │    │    │   │   │    │  »
     c_7: ──────────────────────────────────────────┼────┼────┼───┼───┼────┼──»
                                                  ┌─┴─┐  │    │   │   │    │  »
  zero_0: ────────────────────────────────────────┤ X ├──┼────X───┼───■────┼──»
                                                  └───┘┌─┴─┐      │        │  

### Simulate

In [242]:
# Run and get counts
result = simulator.run(transpiled).result()
#counts = result.get_counts(circ)

In [240]:
# Transpile for simulator
simulator = Aer.get_backend('aer_simulator')
circ = transpile(circuit, backend=simulator)

# Run and get counts
result = simulator.run(circ).result()
#counts = result.get_counts(circ)

Simulation failed and returned the following error message:
ERROR: Failed to load qobj: Unable to cast Python instance to C++ type (compile in debug mode for details)


In [235]:
#c.measure_all()
# Use Aer's qasm_simulator
simulator = Aer.get_backend('statevector_simulator')

# Execute the circuit on the qasm simulator
job = simulator.run(circuit)

# Grab results from the job
result = job.result()

# Return counts
outputstate = result.get_statevector(circuit, decimals=3)
print(outputstate)

Simulation failed and returned the following error message:
ERROR: Failed to load qobj: Unable to cast Python instance to C++ type (compile in debug mode for details)


QiskitError: 'Data for experiment "circuit-2395" could not be found.'